In [1]:
import os
from groq import Groq

In [ ]:
client = Groq(api_key="enter your api key here")

In [19]:
def extract_symptoms(user_input):
    prompt = (
        "You are a medical data extractor. From the following user input, extract: "
        "1. Specific symptoms (e.g., fever, headache). "
        "2. Duration, if mentioned (e.g., two days). "
        "3. Severity, if described (e.g., mild, severe). "
        "Return the result in this format:\n"
        "- Symptoms: [list symptoms]\n"
        "- Duration: [duration or 'not specified']\n"
        "- Severity: [severity or 'not specified']\n\n"
        f"User input: '{user_input}'"
    )
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
            temperature=0.2  # Low temperature for structured output
        )
        
        # Parse the response
        response_text = response.choices[0].message.content.strip()
        symptoms, duration, severity = ["unknown symptom"], "not specified", "not specified"
        
        for line in response_text.split("\n"):
            if line.startswith("- Symptoms:"):
                symptoms = line.split(": ")[1].strip("[]").split(", ")
            elif line.startswith("- Duration:"):
                duration = line.split(": ")[1].strip()
            elif line.startswith("- Severity:"):
                severity = line.split(": ")[1].strip()
        
        return {
            "Symptoms": symptoms,
            "Duration": duration,
            "Severity": severity
        }
    except Exception as e:
        print(f"Error extracting symptoms: {e}")
        return {"Symptoms": ["unknown"], "Duration": "not specified", "Severity": "not specified"}

In [20]:
# Stage 2: Health Summary
def generate_health_summary(symptom_data, user_input):
    prompt = (
        "You are a friendly health assistant. Using the extracted symptom data below, "
        "create a concise, personalized health summary for the user. Include: "
        "1. A restatement of their symptoms in natural language. "
        "2. Possible conditions (general, e.g., 'could be a cold or stress'). "
        "3. Simple, actionable next steps (e.g., rest, see a doctor if needed). "
        "Respond in a conversational tone.\n\n"
        f"Symptom Data:\n"
        f"- Symptoms: {', '.join(symptom_data['Symptoms'])}\n"
        f"- Duration: {symptom_data['Duration']}\n"
        f"- Severity: {symptom_data['Severity']}\n"
        f"Original input: '{user_input}'"
    )
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300,
            temperature=0.8  # Higher temperature for natural tone
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Sorry, I couldn’t generate a summary right now. Try again or consult a doctor."

In [21]:
# Define the main script with a loop for multiple inputs
def main():
    while True:
        user_input = input("Describe your symptoms (or type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        
        # Stage 1: Extract symptoms
        symptom_data = extract_symptoms(user_input)
        print("\nExtracted Data:", symptom_data)  # For debugging
        
        # Stage 2: Generate health summary
        health_summary = generate_health_summary(symptom_data, user_input)
        print("\nHealth Summary:")
        print(health_summary)
        print("-" * 50)

# Run the script
if __name__ == "__main__":
    main()

Describe your symptoms (or type 'exit' to quit):  RASH ON HANDS



Extracted Data: {'Symptoms': ['rash'], 'Duration': 'not specified', 'Severity': 'not specified'}

Health Summary:
It sounds like you're experiencing a rash on your hands, and I'm here to help you figure out what might be going on. 

You've got a rash on your hands, which can be pretty uncomfortable and worrisome. This could be a sign of a skin irritation, an allergic reaction, or possibly even a viral or bacterial infection - it could be something like a cold, an allergic reaction, or even stress that's causing it.

For now, try to keep the affected area clean and moisturized to help soothe any discomfort. If the rash persists, worsens, or is accompanied by other symptoms like fever or difficulty breathing, it's a good idea to consult with a doctor to get a proper diagnosis and treatment. They can help you determine the cause and recommend the best course of action.

Take care of yourself and feel free to reach out if you have any more questions or concerns.
--------------------------

Describe your symptoms (or type 'exit' to quit):  EXIT


Goodbye!
